In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [4]:
from collections import Counter, OrderedDict
import collections

In [5]:
import joblib


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [7]:
import re

In [8]:
from textblob import TextBlob

Based on https://www.kaggle.com/venomsnake/analyzing-sarcasm-comprehensive-eda-detection and https://github.com/surajr/SarcasmDetection

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df_isarcasm=pd.read_csv("/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv")

In [12]:
df_isarcasm_new=df_isarcasm.loc[df_isarcasm['sarcastic']==1]

In [13]:
df_isarcasm_new

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
862,862,yo @claires do yall do hysterectomies?,1,"Claires, you should not do full hysterectomies.",1.0,0.0,0.0,0.0,0.0,1.0
863,863,@JacobWohlReport Do I need to aquire a wife be...,1,A lot of people don't have a wife.,1.0,0.0,0.0,0.0,0.0,1.0
864,864,I get a lot of boy who cried wolf vibes from t...,1,The red cross is always needy.,0.0,1.0,0.0,0.0,0.0,0.0
865,865,Update: holding hands with your mom and walkin...,1,Holding hands with your parent while walking a...,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_isarcasm_new=df_isarcasm_new[['tweet','sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']]

In [79]:
train, validate, test = \
              np.split(df_isarcasm_new.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm_new)), int(.8*len(df_isarcasm_new))])

In [81]:
train_new=pd.concat([train, validate], ignore_index=True)

In [17]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [18]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [19]:
import spacy

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [22]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [23]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [24]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 3.6 MB/s 


In [25]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [26]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [27]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [29]:
import csv

In [30]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [31]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [32]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [33]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [34]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [35]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [36]:
porter = nltk.PorterStemmer()


In [37]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [38]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [82]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train_new['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [40]:
!pip install emoji

     |████████████████████████████████| 170 kB 27.6 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=2ec351fa9273c061434f731b6bb6740838e6946572b711103264152acc3c4d9c
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [41]:
from emoji import UNICODE_EMOJI



In [42]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [43]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [44]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [83]:
train_new.dropna(inplace=True)

In [84]:
train_new

,tweet,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,All the shade i have been hearing about Ben Pl...,1.0,0.0,0.0,0.0,0.0,0.0
1,you would think the odds of me sitting next to...,1.0,0.0,0.0,0.0,0.0,0.0
2,Billy Gunn sorta relevant for the first time i...,1.0,0.0,0.0,1.0,0.0,0.0
3,"I see Dettol, Toilet Duck and Zoflora are all ...",1.0,0.0,0.0,0.0,0.0,1.0
4,@catboychika Aren't you happy I gave you less ...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
688,omg my ships are gonna love what taylor swift ...,1.0,0.0,0.0,0.0,0.0,0.0
689,Maybe if I watched suits in year 12 I’d be stu...,1.0,0.0,0.0,0.0,0.0,0.0
690,"Me: ""Did you you find everything you were look...",1.0,0.0,0.0,0.0,0.0,1.0
691,Broke: doing assignments in order of due date\...,1.0,0.0,0.0,0.0,0.0,0.0


In [58]:
train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[0].values

array([1., 0., 0., 0., 0., 0.])

In [85]:
import re
featureset=[]
for i in range(0,len(train_new)):
    # if i==2774or i==2775:
    #   continue
    temp = str(train_new["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train_new[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values, train_new["tweet"][i]))

In [86]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [87]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [88]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0.200000,0.540000,0.204833,0.196862,0.007971,0.000000,0.00000,0.281642,0.169893,0.111748,0.200000,0.540000,1,0.956739
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0.020833,0.156250,0.343094,0.245032,0.098062,0.020833,0.15625,0.274382,0.246909,0.027473,0.000000,0.000000,0,0.947632
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00000,0.229167,0.000000,0.229167,0.250000,0.333333,0,0.875625
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,-0.033333,0.000000,0.046875,0.036458,0.010417,-0.033333,0.00000,0.064560,0.072802,-0.008242,0.000000,0.000000,1,0.917083
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,-0.483333,0.483333,0.000000,0.000000,0.000000,0.000000,0.00000,0.041667,0.316667,-0.275000,-0.483333,0.483333,0,0.889444


In [89]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [90]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [91]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [92]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [93]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [94]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [95]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [96]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [97]:
result['entities']=result['text'].apply(entity_wordcloud)

In [98]:
result['count_profane']=result['text'].apply(count_profane_words)

In [99]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,All the shade i have been hearing about Ben Pl...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16.0,3.0,8.0,0.0,0.486475,0.366756,0.119719,0.200000,0.540000,0.204833,0.196862,0.007971,0.000000,0.00000,0.281642,0.169893,0.111748,0.200000,0.540000,1,0.956739,True,1,3,0.680000,2,4,4,2,Ben Platt @DearEvanHansen Grease 1,0
1,you would think the odds of me sitting next to...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,6.0,5.0,0.0,0.617476,0.491941,0.125535,0.020833,0.156250,0.343094,0.245032,0.098062,0.020833,0.15625,0.274382,0.246909,0.027473,0.000000,0.000000,0,0.947632,True,0,0,0.451389,4,0,3,5,three,1
2,Billy Gunn sorta relevant for the first time i...,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0]",6.0,1.0,0.0,0.0,0.229167,0.000000,0.229167,0.250000,0.333333,0.000000,0.000000,0.000000,0.000000,0.00000,0.229167,0.000000,0.229167,0.250000,0.333333,0,0.875625,False,0,0,0.616667,0,2,0,2,Billy Gunn first years,0
3,"I see Dettol, Toilet Duck and Zoflora are all ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]",7.0,0.0,5.0,1.0,0.111435,0.109261,0.002175,-0.033333,0.000000,0.046875,0.036458,0.010417,-0.033333,0.00000,0.064560,0.072802,-0.008242,0.000000,0.000000,1,0.917083,True,0,0,0.000000,3,5,1,0,Dettol Toilet Duck Zoflora,0
4,@catboychika Aren't you happy I gave you less ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5.0,1.0,3.0,1.0,0.041667,0.316667,-0.275000,-0.483333,0.483333,0.000000,0.000000,0.000000,0.000000,0.00000,0.041667,0.316667,-0.275000,-0.483333,0.483333,0,0.889444,True,0,0,0.533333,2,0,3,2,,0


In [100]:
len(test)

174

In [101]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), test[['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']].loc[i].values,test['tweet'][i]))

In [102]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [103]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [104]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,-0.775758,0.00,0.000000,0.000000,0.791667,-0.791667,0.00,0.000000,0.095455,0.079545,0.015909,0.0,0.0,0,0.111667
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,-0.104167,0.00,0.000000,0.000000,0.010417,-0.010417,0.00,0.000000,0.000000,0.104167,-0.104167,0.0,0.0,0,0.502500
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.791667,0.25,0.347917,0.979167,0.250000,0.729167,0.25,0.347917,0.187500,0.125000,0.062500,0.0,0.0,0,0.072198
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,-0.917411,-0.10,0.400000,0.229167,0.645833,-0.416667,-0.10,0.350000,0.262649,0.763393,-0.500744,-0.1,0.5,1,0.167500
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.125000,0.20,0.200000,0.000000,0.000000,0.000000,0.40,0.400000,0.208333,0.083333,0.125000,0.0,0.0,0,0.193675


In [105]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [106]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [107]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [108]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [109]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [110]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [111]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [112]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [113]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [114]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [115]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,can't wait til a bunch of pudgy suburban dads ...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,3.0,1.0,0.095455,0.871212,-0.775758,0.00,0.000000,0.000000,0.791667,-0.791667,0.00,0.000000,0.095455,0.079545,0.015909,0.0,0.0,0,0.111667,True,0,0,0.000000,3,0,0,1,the day,0
1,Why are the side effects of antidepressants so...,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]",2.0,2.0,1.0,0.0,0.000000,0.104167,-0.104167,0.00,0.000000,0.000000,0.010417,-0.010417,0.00,0.000000,0.000000,0.104167,-0.104167,0.0,0.0,0,0.502500,True,0,0,0.000000,1,0,0,2,,0
2,I love to pick my modules on a first come firs...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",9.0,5.0,5.0,1.0,1.166667,0.375000,0.791667,0.25,0.347917,0.979167,0.250000,0.729167,0.25,0.347917,0.187500,0.125000,0.062500,0.0,0.0,0,0.072198,True,0,0,0.502778,4,1,2,4,first first my entire year 1995,0
3,@RealCandaceO Ok so when does the anger and bi...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",6.0,3.0,1.0,2.0,0.491815,1.409226,-0.917411,-0.10,0.400000,0.229167,0.645833,-0.416667,-0.10,0.350000,0.262649,0.763393,-0.500744,-0.1,0.5,1,0.167500,True,0,1,0.350000,3,0,1,0,,0
4,@AvatarOdin @aubviouslynot sounds like the ame...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4.0,2.0,1.0,1.0,0.208333,0.083333,0.125000,0.20,0.200000,0.000000,0.000000,0.000000,0.40,0.400000,0.208333,0.083333,0.125000,0.0,0.0,0,0.193675,False,0,0,0.000000,1,2,0,1,american,0


In [116]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'entities', 'count_profane'],
      dtype='object')

In [117]:
result_test.dropna(inplace=True)

In [118]:
result.dropna(inplace=True)

In [119]:
x_train,y_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result[['label']]

In [120]:
x_test,y_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result_test[['label']]

In [127]:
y_test

,label
0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...
169,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
170,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
171,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
172,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [121]:
from sklearn.multioutput import MultiOutputClassifier

In [123]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [130]:
def print_statistics(y, y_pred):
    accuracy = hamming_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred,target_names=['sarcasm', 'irony',
       'satire', 'understatement', 'overstatement', 'rhetorical_question']))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'estimator__C': C_range}
    linear_svm = MultiOutputClassifier(LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2'))
    # print(linear_svm.get_params().keys())
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'estimator__kernel': ['rbf'], 'estimator__C': C_range, 'estimator__gamma': gamma_range}
    nonlinear_svm = MultiOutputClassifier(SVC(class_weight=class_ratio))
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'estimator__C': C_range}
    log_regr = MultiOutputClassifier(LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2'))
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = MultiOutputClassifier(LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2'))
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = MultiOutputClassifier(LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2'))
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = MultiOutputClassifier(RandomForestClassifier(n_estimators=400, random_state=11))
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =MultiOutputClassifier(XGBClassifier(max_depth=6, n_estimators=1000))
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = MultiOutputClassifier(LinearSVC(C=0.1, penalty='l2'))
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'))),('svm_linear',MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2'))),('rf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10, random_state=42)))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=MultiOutputClassifier(LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42))
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [ ]:
linear_svm_grid(x_train,np.array(y_train).astype(int),x_test,np.array(y_test).astype(int),class_ratio='balanced')


In [155]:
y_train_l=[]
for i in range(0,len(y_train.to_numpy())):
  y_train_l.append(y_train.to_numpy()[i][0].astype(int))
y_train_l=np.array(y_train_l)


In [156]:
y_test_l=[]
for i in range(0,len(y_test.to_numpy())):
  y_test_l.append(y_test.to_numpy()[i][0].astype(int))
y_test_l=np.array(y_test_l)

In [158]:
import warnings
warnings.filterwarnings('ignore')

In [159]:
linear_svm_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.647
Precision: 0.667
Recall: 0.664
F_score: 0.659

                     precision    recall  f1-score   support

            sarcasm       0.81      0.88      0.84       138
              irony       0.31      0.22      0.26        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.73      0.44      0.55        25

          micro avg       0.72      0.66      0.69       211
          macro avg       0.31      0.26      0.27       211
       weighted avg       0.67      0.66      0.66       211
        samples avg       0.71      0.68      0.68       211



In [160]:
nonlinear_svm_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.692
Precision: 0.562
Recall: 0.645
F_score: 0.582

                     precision    recall  f1-score   support

            sarcasm       0.79      0.98      0.88       138
              irony       0.25      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.78      0.64      0.70       211
          macro avg       0.17      0.17      0.15       211
       weighted avg       0.56      0.64      0.58       211
        samples avg       0.78      0.69      0.72       211



In [161]:
logistic_regression_grid(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.427
Precision: 0.632
Recall: 0.573
F_score: 0.569

                     precision    recall  f1-score   support

            sarcasm       0.80      0.57      0.66       138
              irony       0.22      0.44      0.29        36
             satire       0.12      0.80      0.21         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.06      0.67      0.11         6
rhetorical_question       0.59      0.76      0.67        25

          micro avg       0.40      0.57      0.47       211
          macro avg       0.30      0.54      0.32       211
       weighted avg       0.63      0.57      0.57       211
        samples avg       0.45      0.56      0.48       211



In [162]:
linear_svm(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.351
Precision: 0.623
Recall: 0.578
F_score: 0.565

                     precision    recall  f1-score   support

            sarcasm       0.78      0.57      0.66       138
              irony       0.19      0.39      0.25        36
             satire       0.09      0.80      0.16         5
     understatement       0.03      1.00      0.05         1
      overstatement       0.08      0.83      0.14         6
rhetorical_question       0.63      0.76      0.69        25

          micro avg       0.35      0.58      0.43       211
          macro avg       0.30      0.73      0.33       211
       weighted avg       0.62      0.58      0.57       211
        samples avg       0.37      0.57      0.43       211



In [163]:
logistic_regression(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.328
Precision: 0.618
Recall: 0.555
F_score: 0.542

                     precision    recall  f1-score   support

            sarcasm       0.79      0.53      0.63       138
              irony       0.20      0.47      0.28        36
             satire       0.06      0.40      0.11         5
     understatement       0.03      1.00      0.05         1
      overstatement       0.07      0.83      0.13         6
rhetorical_question       0.51      0.76      0.61        25

          micro avg       0.33      0.55      0.41       211
          macro avg       0.28      0.67      0.30       211
       weighted avg       0.62      0.55      0.54       211
        samples avg       0.35      0.55      0.41       211



In [164]:
random_forest(x_train,y_train_l,x_test,y_test_l,class_ratio='balanced')


       Random Forest

Accuracy: 0.722
Precision: 0.622
Recall: 0.687
F_score: 0.629

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.88      0.28      0.42        25

          micro avg       0.80      0.69      0.74       211
          macro avg       0.28      0.21      0.22       211
       weighted avg       0.62      0.69      0.63       211
        samples avg       0.80      0.73      0.75       211



In [165]:
xg_boost(x_train,y_train_l,x_test,y_test_l)


             XGBoost

Accuracy: 0.701
Precision: 0.640
Recall: 0.692
F_score: 0.649

                     precision    recall  f1-score   support

            sarcasm       0.80      0.96      0.87       138
              irony       0.14      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.80      0.48      0.60        25

          micro avg       0.77      0.69      0.73       211
          macro avg       0.29      0.25      0.25       211
       weighted avg       0.64      0.69      0.65       211
        samples avg       0.76      0.72      0.73       211



In [169]:
gradient_boosting(x_train,y_train_l,x_test,y_test_l)


   Gradient Boosting

Accuracy: 0.618
Precision: 0.655
Recall: 0.626
F_score: 0.636

                     precision    recall  f1-score   support

            sarcasm       0.80      0.82      0.81       138
              irony       0.30      0.19      0.24        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.67      0.48      0.56        25

          micro avg       0.72      0.63      0.67       211
          macro avg       0.30      0.25      0.27       211
       weighted avg       0.66      0.63      0.64       211
        samples avg       0.67      0.65      0.65       211



In [170]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(result['text'])
char_vectorizer.fit(result['text'])


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [171]:
train_word_features = word_vectorizer.transform(result['text'])
train_char_features = char_vectorizer.transform(result['text'])

In [172]:
test_word_features = word_vectorizer.transform(result_test['text'])
test_char_features = char_vectorizer.transform(result_test['text'])

In [173]:
from scipy.sparse import hstack, csr_matrix


In [174]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [175]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [176]:
linear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.686
Precision: 0.668
Recall: 0.668
F_score: 0.659

                     precision    recall  f1-score   support

            sarcasm       0.80      0.89      0.84       138
              irony       0.30      0.17      0.21        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.80      0.48      0.60        25

          micro avg       0.73      0.67      0.70       211
          macro avg       0.32      0.26      0.28       211
       weighted avg       0.67      0.67      0.66       211
        samples avg       0.73      0.70      0.71       211



In [177]:
nonlinear_svm_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


       Nonlinear SVM

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [178]:
logistic_regression_grid(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.682
Precision: 0.673
Recall: 0.668
F_score: 0.663

                     precision    recall  f1-score   support

            sarcasm       0.81      0.88      0.84       138
              irony       0.29      0.19      0.23        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.80      0.48      0.60        25

          micro avg       0.72      0.67      0.69       211
          macro avg       0.32      0.26      0.28       211
       weighted avg       0.67      0.67      0.66       211
        samples avg       0.73      0.70      0.70       211



In [179]:
linear_svm(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


          Linear SVM

Accuracy: 0.467
Precision: 0.647
Recall: 0.621
F_score: 0.612

                     precision    recall  f1-score   support

            sarcasm       0.80      0.65      0.72       138
              irony       0.23      0.39      0.29        36
             satire       0.08      0.40      0.13         5
     understatement       0.05      1.00      0.10         1
      overstatement       0.12      0.83      0.20         6
rhetorical_question       0.66      0.76      0.70        25

          micro avg       0.45      0.62      0.52       211
          macro avg       0.32      0.67      0.36       211
       weighted avg       0.65      0.62      0.61       211
        samples avg       0.49      0.62      0.53       211



In [180]:
logistic_regression(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')


 Logistic Regression

Accuracy: 0.359
Precision: 0.623
Recall: 0.559
F_score: 0.552

                     precision    recall  f1-score   support

            sarcasm       0.80      0.54      0.65       138
              irony       0.21      0.47      0.29        36
             satire       0.03      0.20      0.06         5
     understatement       0.04      1.00      0.07         1
      overstatement       0.07      0.83      0.14         6
rhetorical_question       0.53      0.76      0.62        25

          micro avg       0.35      0.56      0.43       211
          macro avg       0.28      0.63      0.30       211
       weighted avg       0.62      0.56      0.55       211
        samples avg       0.38      0.55      0.43       211



In [181]:
random_forest(train_features,y_train_l,test_features,y_test_l,class_ratio='balanced')



       Random Forest

Accuracy: 0.707
Precision: 0.519
Recall: 0.654
F_score: 0.579

                     precision    recall  f1-score   support

            sarcasm       0.79      1.00      0.88       138
              irony       0.00      0.00      0.00        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.00      0.00      0.00        25

          micro avg       0.79      0.65      0.72       211
          macro avg       0.13      0.17      0.15       211
       weighted avg       0.52      0.65      0.58       211
        samples avg       0.79      0.71      0.74       211



In [182]:
gradient_boosting(train_features,y_train_l,test_features,y_test_l)


   Gradient Boosting

Accuracy: 0.682
Precision: 0.597
Recall: 0.664
F_score: 0.616

                     precision    recall  f1-score   support

            sarcasm       0.79      0.96      0.87       138
              irony       0.14      0.03      0.05        36
             satire       0.00      0.00      0.00         5
     understatement       0.00      0.00      0.00         1
      overstatement       0.00      0.00      0.00         6
rhetorical_question       0.47      0.28      0.35        25

          micro avg       0.73      0.66      0.70       211
          macro avg       0.23      0.21      0.21       211
       weighted avg       0.60      0.66      0.62       211
        samples avg       0.75      0.70      0.71       211

